In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sb 
import matplotlib.pyplot as plt 
from matplotlib.ticker import FuncFormatter

In [ ]:
def make_all_scatterplot(fairness_df, filename=None):

    sb.set_style("white")
    sb.set(font_scale=1.25)
    color_map = {
    "Rescore": "#1f77b4",   
    "Copeland": "#ff7f0e", 
    "Borda": "#2ca02c",      
    }
    agents = fairness_df['agents'].unique()
    n_agents = len(agents)
    

    fig, axes = plt.subplots(1, n_agents, figsize=(6 * n_agents, 5), sharey=True)
    
    if n_agents == 1:
        axes = [axes]  
    
    legend_handles = None 

    for ax, agent in zip(axes, agents):

        agent_data = fairness_df[fairness_df['agents'] == agent]
        
        scatterplot = sb.scatterplot(
            ax=ax,
            x='fairness_norm',
            y='mean_ndcg',
            data=agent_data,
            style="allocation",
            hue="choice",
            s=130,
            markers={"Lottery": "^", "Weighted": "X", "Least Fair": "o"},
            palette=color_map
            
        )
        

        if legend_handles is None:
            legend_handles, legend_labels = ax.get_legend_handles_labels()
            #legend_labels = ['Choice Rule', 'Rescore', 'Borda', 'Copeland', 'Allocation', 'Weighted', 'Lottery', 'Least Fair']
            # ['choice', 'weightedscoring', 'RuleBorda', 'RuleCopeland', 'allocation', 'weighted', 'product', 'least']
        
        
        ax.legend_.remove()

        #ax.axhline(y=base_ndcg, linestyle="dashed", color="gray")
        # ax.set_title(f"Agent: {agent}", fontsize=14)
        ax.set_xlabel("Normalized Fairness", fontsize=18)
        if ax == axes[0]:  
            ax.set_ylabel("NDCG", fontsize=18)
        else:
            ax.set_ylabel("")
        ax.tick_params(axis="both", labelsize=18)
    

    fig.legend(legend_handles, legend_labels, fontsize=18, loc="center left", bbox_to_anchor=(1, 0.5), markerscale=1.2)

    # if version == 'fairness_norm':
    #     ax.scatter(dataset_x, dataset_y, color="gray", marker="s", s=100, label="Baseline")
    # else:
    #     ax.axhline(y=dataset_y, color="gray", linestyle="dotted", linewidth=2, label="Baseline")
    #ax.axhline(y=dataset_y, color="gray", linestyle="dotted", linewidth=2, label="Baseline")

    plt.tight_layout(rect=[0, 0, 1, 0.8]) 
    if filename is not None:
        plt.savefig(filename)
    plt.show()

In [ ]:
custom_palette = {
    "Weighted": "#CA0F9A",  
    "Lottery": "#D5E31D", 
    "Least Fair": "#354690" 
}

In [ ]:
movies = pd.read_csv('movie_weights_full.csv')
music = pd.read_csv('music_weights_full.csv')

In [ ]:
#clean dataframes
movies['weight'] = movies['weight'].str.split(" ").str[0]
movies['weight'] = movies['weight'].astype(float)
movies['coverage'] = movies['coverage'].clip(upper=1)
music['coverage'] = music['coverage'].clip(upper=1)
movies['proportional_fairness_women'] = movies['proportional_fairness_women'].clip(upper=1)
music['proportional_fairness_women'] = music['proportional_fairness_women'].clip(upper=1)
movies['proportional_fairness_non-en'] = movies['proportional_fairness_non-en'].clip(upper=1)

replacements = {
    'RuleBorda': 'Borda',
    'RuleCopeland': 'Copeland',
    'Product': 'Weighted',
    'leastFair': 'Least Fair',
    'lottery': 'Lottery'
}

movies = movies.replace(replacements)
music = music.replace(replacements)

In [ ]:
#visualize ndcg trade-off with recommender weight for movies data
for agent in movies["agents"].unique():
    movies_agent = movies[movies["agents"] == agent]
    if agent == "group":
        movies['fairness_norm'] = movies['proportional_fairness_women']
    if agent == "individual":
        movies['fairness_norm'] = movies['coverage']
    if agent == "all":
        movies['fairness_norm'] = (movies['proportional_fairness_women']**(1/2)+movies['proportional_fairness_non-en']**(1/2)+movies['coverage']**(1/2))/3
    
    
    g = sb.FacetGrid(movies_agent, col="choice", col_wrap=3, height=4, sharex=True, sharey=True)

    g.map_dataframe(sb.lineplot, x="weight", y="mean_ndcg", hue="allocation", marker="o", palette=custom_palette)
    g.add_legend(title="Allocation Type")
    g.figure.suptitle(f"NDCG vs fairness Trade-off for Agent: {agent}", fontsize=16, fontweight="bold")
    g.figure.subplots_adjust(top=0.85, hspace=0.3)
    g.map(plt.axhline, y=0.1383, ls='--')
    for ax, title in zip(g.axes.flat, movies_agent["choice"].unique()):
        ax.set_title(f"Choice Mechanism: {title}", fontsize=12, fontweight="bold")
    
    plt.show()

In [ ]:
#visualize ndcg trade-off with recommender weight for music data
for agent in music["agents"].unique():
    music_agent = music[music["agents"] == agent]
    if agent == "group":
        music['fairness_norm'] = music['proportional_fairness_women']
    if agent == "individual":
        music['fairness_norm'] = music['coverage']
    if agent == "all":
        music['fairness_norm'] = (music['proportional_fairness_women']**(1/2)+music['coverage']**(1/2))/2
    
    g = sb.FacetGrid(music_agent, col="choice", col_wrap=3, height=4, sharex=True, sharey=True)
    plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{x:.3f}"))
    g.map_dataframe(sb.lineplot, x="weight", y="mean_ndcg", hue="allocation", marker="o", palette=custom_palette)

    g.add_legend(title="Allocation Type")
    g.figure.suptitle(f"NDCG vs fairness Trade-off for Agent: {agent}", fontsize=16, fontweight="bold")
    g.figure.subplots_adjust(top=0.85, hspace=0.3)
    g.map(plt.axhline, y=0.09899, ls='--')
    
    for ax, title in zip(g.axes.flat, music_agent["choice"].unique()):
        ax.set_title(f"Choice Mechanism: {title}", fontsize=12, fontweight="bold")
    
    plt.show()

In [ ]:
# grab the lowest recommender weight experiment that maintains the 15% cutoff for movies
filtered_movie = movies[movies['mean_ndcg'] >= 0.1383]

result_movie = (
    filtered_movie
    .sort_values('weight') 
    .groupby(['agents', 'choice', 'allocation'], as_index=False)
    .first()  
)

In [ ]:
# grab the lowest recommender weight experiment that maintains the 15% cutoff for music
filtered_music = music[music['mean_ndcg'] >= 0.08857]

result_music = (
    filtered_music
    .sort_values('weight') 
    .groupby(['agents', 'choice', 'allocation'], as_index=False)
    .first()  
)

In [ ]:
#save this result as csv 
# result_movie.to_csv("movie_15.csv")
# result_music.to_csv("music_15.csv")

In [ ]:
#Separate out different agent conditions
group_movie = result_movie[result_movie["agents"]=='group']
all_movie = result_movie[result_movie["agents"]=='all']
ind_movie = result_movie[result_movie["agents"]=='individual']

group_music = result_music[result_music["agents"]=='group']
all_music = result_music[result_music["agents"]=='all']
ind_music = result_music[result_music["agents"]=='individual']

In [ ]:
#create fairness norm columns
all_music['fairness_norm'] = (all_music['proportional_fairness_women']**(1/2)+all_music['coverage']**(1/2))/2
group_movie['fairness_norm'] = (group_movie['proportional_fairness_women']**(1/2)+group_movie['proportional_fairness_non-en']**(1/2))/2
all_movie['fairness_norm'] = (all_movie['proportional_fairness_women']**(1/2)+all_movie['proportional_fairness_non-en']**(1/2)+all_movie['coverage']**(1/2))/3


In [ ]:
make_all_scatterplot(all_movie)

In [ ]:
make_all_scatterplot(all_music)